In [3]:
import chainer
import chainer.links as L
import chainer.functions as F
from chainer import iterators, training, optimizers, serializers
from chainer.training import extensions

/Users/ootaryuunosuke/.pyenv/versions/anaconda3-5.1.0/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
# cifar10の読み込み
def load_cifar10():
    (train, test) = chainer.datasets.get_cifar10()
    return train, test

# 特定クラスラベル画像の取得
def get_one_label_data(class_label, dataset):
    new_data = []
    for data in dataset:
        label = data[1]
        if label == class_label:
            new_data.append(data)
    return new_data

# 特定クラスラベルのtrain, testデータセットを取得
def get_one_label_dataset(class_label, train, test):
    train = get_one_label_data(class_label, train)
    test = get_one_label_data(class_label, test)
    return train, test

# 特定クラスラベルリストのtrain, testデータセットを取得
def get_specific_label_dataset(class_label_list, train, test):
    new_train = []
    new_test = []
    for label in class_label_list:
        got_train, got_test = get_one_label_dataset(label, train, test)
        new_train += got_train
        new_test += got_test
    return new_train, new_test

In [6]:
class_label_list = [1,2,7]

train, test = load_cifar10()
train, test = get_specific_label_dataset(class_label_list, train, test)

In [16]:
class Cifar10Model(chainer.Chain):

    def __init__(self, n_out=10):
        super(Cifar10Model,self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(3, 32, 3, pad=1)
            self.conv2 = L.Convolution2D(32, 32, 3, pad=1)
            self.conv3 = L.Convolution2D(32, 32, 3, pad=1)
            self.conv4 = L.Convolution2D(32, 32, 3, pad=1)
            self.conv5 = L.Convolution2D(32, 32, 3, pad=1)
            self.conv6 = L.Convolution2D(32, 32, 3, pad=1)
            self.l1 = L.Linear(512, 512)
            self.l2 = L.Linear(512, n_out)

    def __call__(self, x):
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        h = F.relu(self.conv3(h))
        h = F.max_pooling_2d(F.relu(self.conv4(h)), 2)
        h = F.relu(self.conv5(h))
        h = F.max_pooling_2d(F.relu(self.conv6(h)), 2)
        h = F.dropout(F.relu(self.l1(h)))
        return self.l2(h)

In [22]:
class MyNet(chainer.Chain):

    def __init__(self, n_out=10):
        super(MyNet, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(None, 32, 3, 3, 1)
            self.conv2 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv3 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv4 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv5 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv6 = L.Convolution2D(32, 32, 3, 3, 1)
            self.fc1 = L.Linear(None, 512)
            self.fc2 = L.Linear(512, n_out)

    def __call__(self, x):
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        h = F.relu(self.conv3(h))
        h = F.max_pooling_2d(F.relu(self.conv4(h)), 2)
        h = F.relu(self.conv5(h))
        h = F.max_pooling_2d(F.relu(self.conv6(h)), 2)
        h = F.dropout(F.relu(self.fc1(h)))
        h = self.fc2(h)
        return h

In [4]:
class Cifar10(chainer.Chain):

    def __init__(self, n_out=3):
        super(Cifar10, self).__init__()
        with self.init_scope():
            self.conv1 = L.Convolution2D(None, 32, 3, 3, 1)
            self.conv2 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv3 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv4 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv5 = L.Convolution2D(32, 32, 3, 3, 1)
            self.conv6 = L.Convolution2D(32, 32, 3, 3, 1)
            self.fc1 = L.Linear(None, 512)
            self.fc2 = L.Linear(512, n_out)

    def __call__(self, x):
        h = F.relu(self.conv1(x))
        h = F.max_pooling_2d(F.relu(self.conv2(h)), 2)
        h = F.relu(self.conv3(h))
        h = F.max_pooling_2d(F.relu(self.conv4(h)), 2)
        h = F.relu(self.conv5(h))
        h = F.max_pooling_2d(F.relu(self.conv6(h)), 2)
        h = F.dropout(F.relu(self.fc1(h)))
        h = self.fc2(h)
        return h

In [23]:
batch_size = 128
epoch = 1

#model = L.Classifier(Cifar10Model())
model = L.Classifier(MyNet())
optimizer = chainer.optimizers.Adam()
optimizer.setup(model)

train_iter = chainer.iterators.SerialIterator(train, batch_size)
test_iter = chainer.iterators.SerialIterator(test, batch_size ,repeat=False, shuffle=False)

updater = training.StandardUpdater(train_iter, optimizer, device=-1)
trainer = training.Trainer(updater, (epoch, 'epoch'), out="logs")
trainer.extend(extensions.Evaluator(test_iter, model, device=-1))
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport( ['epoch', 'main/loss', 'validation/main/loss', 'main/accuracy', 'validation/main/accuracy']))                                                                                          
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy
1           1.99875     1.76195               0.228001       0.324862                  
